In [1]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from tqdm import notebook
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
from pymystem3 import Mystem
m = Mystem()



In [2]:
data = pd.read_excel('CRA_train_1200.xlsx')

In [4]:
data['Категория'].value_counts(normalize=True)

A      0.321667
BBB    0.225000
AA     0.165833
AAA    0.127500
BB     0.113333
B      0.036667
C      0.010000
Name: Категория, dtype: float64

In [5]:
data['Уровень рейтинга'].value_counts(normalize=True)

AAA     0.127500
A-      0.125000
A+      0.105000
BBB+    0.095833
A       0.091667
BBB     0.075000
AA      0.061667
AA+     0.059167
BBB-    0.054167
BB+     0.047500
AA-     0.045000
BB      0.041667
BB-     0.024167
B+      0.016667
B       0.010000
B-      0.010000
C       0.010000
Name: Уровень рейтинга, dtype: float64

In [6]:
# import fasttext.util
# from gensim.models import FastText
# import gensim.downloader as api
# import pandas as pd

# #fasttext.util.download_model('ru', if_exists='ignore') 
# ft_model = FastText.load_fasttext_format('cc.ru.300.bin')

# data['pr_txt'] = data['pr_txt'].apply(lambda x: str.lower(x))
# data['pr_txt'] = data['pr_txt'].apply(lambda x: re.sub(r'[0-9]+', '', x))
# #data['pr_txt_tokens'] = data['pr_txt'].apply(lambda x: word_tokenize(x))

# #data['pr_txt_tokens'] = data['pr_txt'].apply(lambda x: m.lemmatize(x))

# def get_embedding(sentence):
#     #tokens = word_tokenize(sentence)
#     tokens =  m.lemmatize(sentence)
#     vector = [ft_model.wv[token] for token in tokens if token in ft_model.wv]
#     if vector:
#         return sum(vector) / len(vector)
#     else:
#         return [0] * 300 

# data['embedding'] = data['pr_txt'].apply(get_embedding)

In [7]:
data['pr_txt'] = data['pr_txt'].apply(lambda x: str.lower(x))
data['pr_txt'] = data['pr_txt'].apply(lambda x: re.sub(r'[0-9]+', '', x))
data['pr_txt'] = data['pr_txt'].apply(lambda x: re.sub(r'[^\w\s]','',x))
data['pr_txt_tokens'] = data['pr_txt'].apply(lambda x: word_tokenize(x))

model = Word2Vec(data['pr_txt_tokens'], vector_size=300, window=14, min_count=4, sg=1)

def get_embedding(sentence):
    tokens = m.lemmatize(sentence)
    #tokens = word_tokenize(sentence)
    vector = [model.wv[token] for token in tokens if token in model.wv]
    if vector:
        return sum(vector) / len(vector)
    else:
        return [0] * 300  

data['embedding'] = data['pr_txt'].apply(get_embedding)
model.save('model.kvmodel')

In [8]:
X = pd.DataFrame(data['embedding'].tolist())
y = data[['Категория','Уровень рейтинга']]
features_train, features_test, target_train, target_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
clf = CatBoostClassifier(iterations=800,  
                         depth=5, 
                         learning_rate=0.1,
                         loss_function='MultiClass',
                         custom_metric=['F1'])

train_data = Pool(features_train, label=target_train['Категория'])
clf.fit(train_data)
test_data = Pool(features_test)
predictions1 = clf.predict(test_data)
clf.save_model('model1.cbm')

0:	learn: 1.8905122	total: 146ms	remaining: 1m 56s
1:	learn: 1.8317285	total: 178ms	remaining: 1m 11s
2:	learn: 1.7835090	total: 208ms	remaining: 55.3s
3:	learn: 1.7417610	total: 238ms	remaining: 47.4s
4:	learn: 1.7049061	total: 268ms	remaining: 42.6s
5:	learn: 1.6643310	total: 298ms	remaining: 39.4s
6:	learn: 1.6335467	total: 328ms	remaining: 37.1s
7:	learn: 1.6025715	total: 360ms	remaining: 35.7s
8:	learn: 1.5691607	total: 391ms	remaining: 34.4s
9:	learn: 1.5419143	total: 423ms	remaining: 33.4s
10:	learn: 1.5175261	total: 454ms	remaining: 32.6s
11:	learn: 1.4890722	total: 485ms	remaining: 31.9s
12:	learn: 1.4675376	total: 515ms	remaining: 31.2s
13:	learn: 1.4406295	total: 544ms	remaining: 30.6s
14:	learn: 1.4178028	total: 577ms	remaining: 30.2s
15:	learn: 1.3999662	total: 607ms	remaining: 29.8s
16:	learn: 1.3842153	total: 638ms	remaining: 29.4s
17:	learn: 1.3619439	total: 668ms	remaining: 29s
18:	learn: 1.3485741	total: 698ms	remaining: 28.7s
19:	learn: 1.3275695	total: 729ms	remaini

In [10]:
clf2 = CatBoostClassifier(iterations=500,
                         depth=6, 
                         learning_rate=0.1,  
                         loss_function='MultiClass',
                         custom_metric=['F1'])

train_data = Pool(features_train, label=target_train['Уровень рейтинга'])
clf2.fit(train_data)
test_data = Pool(features_test)
predictions2 = clf2.predict(test_data)
clf2.save_model('model2.cbm')

0:	learn: 2.7791910	total: 127ms	remaining: 1m 3s
1:	learn: 2.7250618	total: 250ms	remaining: 1m 2s
2:	learn: 2.6835572	total: 371ms	remaining: 1m 1s
3:	learn: 2.6446071	total: 491ms	remaining: 1m
4:	learn: 2.6001518	total: 613ms	remaining: 1m
5:	learn: 2.5622132	total: 733ms	remaining: 1m
6:	learn: 2.5194047	total: 852ms	remaining: 1m
7:	learn: 2.4772124	total: 971ms	remaining: 59.7s
8:	learn: 2.4408332	total: 1.1s	remaining: 59.8s
9:	learn: 2.4023042	total: 1.21s	remaining: 59.5s
10:	learn: 2.3697240	total: 1.34s	remaining: 59.4s
11:	learn: 2.3386514	total: 1.46s	remaining: 59.3s
12:	learn: 2.3101314	total: 1.58s	remaining: 59.3s
13:	learn: 2.2817970	total: 1.7s	remaining: 59.1s
14:	learn: 2.2533052	total: 1.82s	remaining: 58.9s
15:	learn: 2.2270715	total: 1.94s	remaining: 58.7s
16:	learn: 2.2006770	total: 2.07s	remaining: 58.9s
17:	learn: 2.1746023	total: 2.19s	remaining: 58.7s
18:	learn: 2.1455778	total: 2.31s	remaining: 58.5s
19:	learn: 2.1187427	total: 2.43s	remaining: 58.4s
20:	

In [11]:
score1 = f1_score(target_test['Категория'],predictions1,average='micro')
score2 = f1_score(target_test['Уровень рейтинга'],predictions2,average='micro')

print("Метрика F1 для целевого признака Категория - ",score1)
print("Метрика F1 для целевого признака Уровень рейтинга - ",score2)
print('целевая метрика:',(score1*0.35)+(score2*0.65))

Метрика F1 для целевого признака Категория -  0.6458333333333334
Метрика F1 для целевого признака Уровень рейтинга -  0.49166666666666664
целевая метрика: 0.545625


In [12]:
word_vector = clf.get_feature_importance()

similar_words = model.wv.similar_by_vector(word_vector, topn=30)

# Print the similar words
for word, score in similar_words:
    print(f'{word}: {score}')

тонны: 0.2575618028640747
полимеров: 0.23292788863182068
пришлось: 0.2326480597257614
сельскохозяйственного: 0.2311207503080368
стальной: 0.22674989700317383
стоимостью: 0.22638192772865295
фармацевтическое: 0.22583208978176117
самообеспеченность: 0.22096851468086243
труб: 0.22030854225158691
румынии: 0.2189483493566513
стали: 0.21858203411102295
бесшовных: 0.21789485216140747
китае: 0.2173893302679062
концентрат: 0.21633602678775787
газетной: 0.21622921526432037
сортамента: 0.21594564616680145
стального: 0.21490882337093353
меди: 0.21460451185703278
калия: 0.21412107348442078
горнорудный: 0.21382462978363037
законтрактована: 0.21264128386974335
полипропилена: 0.21211062371730804
заготовки: 0.20961116254329681
куриное: 0.20937193930149078
реализованной: 0.20902298390865326
коксующегося: 0.2078513205051422
картона: 0.20761068165302277
потребителям: 0.2068701535463333
растениеводства: 0.20653945207595825
производителем: 0.20649921894073486
